# 한국어 word2vec 만들기

In [14]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from konlpy.tag import Okt
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
import json
import os
import chardet

훈련 데이터 전처리

In [15]:
### 전체 파일 사용하는 코드 (수정 필요)
folder = '구어_말뭉치'
## "document" -> "utterance" -> "form" 내용 parsing
utterance_forms = []

def load_data(file_path):
    try:
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                parsed_data = json.load(file)

                for doc in parsed_data['document']:
                    for utterance in doc['utterance']:
                        utterance_forms.append(utterance['form'])

                korean_stopwords = [
                    '이', '그', '저', '것', '수', '등', '들', '여기', '저기', '거기', '너', '나', '우리', '저희', 
                    '그들', '그녀', '그것', '이것', '저것', '무엇', '어떤', '모두', '전부', '한', '에', '가', '를', 
                    '을', '의', '에게', '와', '과', '하고', '다', '부터', '까지', '에서', '에는', '은', '는', '로', 
                    '으로', '만', '말', '하다', '그렇다', '이렇다', '아니다', '이다', '그러나', '그래도', '하지만', 
                    '그러면', '그래서', '그러니까', '또는', '즉', '따라서', '하다', '라고', '이라고', '하면', '라면', 
                    '한다', '된다', '또', '이나', '입니다', '어느', '어떤', '무슨', '한데', '있다', '같다', '예를', 
                    '등을', '등의', '도', '에도', '있습니다', '.', '텐데요', '하는데요', '요', '하는', '되고', '되는', '합니다']

                ## tokenizing
                tokenized_text = []
                # create instance
                okt = Okt()

                # 문장을 형태소 단위로 토크나이징
                for sentence in utterance_forms:
                    tokens = okt.morphs(sentence)
                    # 불용어 제거
                    filtered_tokens = [token for token in tokens if token not in korean_stopwords]
                    tokenized_text.append(filtered_tokens)
        except UnicodeDecodeError:
            with open(file_path, 'rb') as file:
                raw_data = file.read()
                encoding = chardet.detect(raw_data)['encoding']
                if encoding is None:
                    print(f"Unable to determine encoding for {file_path}.")
                    return None
                data_dict = json.loads(raw_data.decode(encoding))
    except FileNotFoundError:
        print(f"{file_path} does not exist.")
        return None
    except json.JSONDecodeError:
        print(f"could not decode JSON from {file_path}")
        return None
    
for file in os.listdir(folder[:3]):
    # os.path.join : 인수에 전달된 2개의 문자열을 결합하여, 1개의 경로로 할 수 있음
    file_path = os.path.join(folder, file)
    load_data(file_path)

FileNotFoundError: [Errno 2] No such file or directory: '구어_'

In [7]:
### 일단 파일 하나만 가지고 해보기
targertJSON = open('./구어_말뭉치/SARW1800000001.json', 'r', encoding='utf-8')
parsed_data = json.load(targertJSON)

## "document" -> "utterance" -> "form" 내용 parsing
utterance_forms = []

for doc in parsed_data['document']:
    for utterance in doc['utterance']:
        utterance_forms.append(utterance['form'])

korean_stopwords = [
    '이', '그', '저', '것', '수', '등', '들', '여기', '저기', '거기', '너', '나', '우리', '저희', 
    '그들', '그녀', '그것', '이것', '저것', '무엇', '어떤', '모두', '전부', '한', '에', '가', '를', 
    '을', '의', '에게', '와', '과', '하고', '다', '부터', '까지', '에서', '에는', '은', '는', '로', 
    '으로', '만', '말', '하다', '그렇다', '이렇다', '아니다', '이다', '그러나', '그래도', '하지만', 
    '그러면', '그래서', '그러니까', '또는', '즉', '따라서', '하다', '라고', '이라고', '하면', '라면', 
    '한다', '된다', '또', '이나', '입니다', '어느', '어떤', '무슨', '한데', '있다', '같다', '예를', '처럼',
    '등을', '등의', '도', '에도', '있습니다', '.', '텐데요', '하는데요', '요', '하는', '되고', '되는', '합니다']

## tokenizing
tokenized_text = []
# create instance
okt = Okt()

# 문장을 형태소 단위로 토크나이징
for sentence in utterance_forms:
    tokens = okt.morphs(sentence)
    # 불용어 제거
    filtered_tokens = [token for token in tokens if token not in korean_stopwords]
    tokenized_text.append(filtered_tokens)

tokenized_text

[['요즘', '추운', '날씨'],
 ['따뜻한', '라테', '한잔', '찾는', '분들'],
 ['많으실'],
 ['라테', '위', '그려진'],
 ['다양한', '라테', '아트', '구경'],
 ['다른', '재미'],
 ['그런데', '최근'],
 ['라테아트', '넘어'],
 ['차가운', '커피', '위'],
 ['색색', '크림'],
 ['그림', '그리는'],
 ['크리', '마트', '인기'],
 ['크리', '마트', '개발', '바리스타', '청년'],
 ['오늘', '꿈', '잡아라', '소개'],
 ['커피', '중', '에서도'],
 ['부드러운', '라테'],
 ['우유', '와의', '궁합', '자랑'],
 ['대표', '적', '겨울', '음룐데'],
 ['특히', '라테', '위', '그려지는'],
 ['라테아트', '눈', '먼저'],
 ['즐거움', '선사', '해줍니다'],
 ['최근', '가장', '기본', '그림', '인'],
 ['로제타', '하트', '외'],
 ['글씨', '쓰기', '각종', '과일'],
 ['빵', '올려', '꾸미기까지'],
 ['다양한', '응용', '아트', '등장', '있는데요'],
 ['중심', '커피', '하얀', '우유', '만으로', '그림', '그리는'],
 ['라테아트', '넘어'],
 ['형형색색', '크림', '그림', '그리는'],
 ['크리', '마트'],
 ['어', '~', '크리', '마트', '일단', '라테아트'],
 ['스킬', '기술', '이름', '아니고요'],
 ['하나', '메뉴', '이름', '으로서'],
 ['우선', '따뜻한', '거', '생각', '많이', '하시는', '분', '많은데'],
 ['무조건', '차갑게만', '만들고', '커피', '포함'],
 ['차가운', '음료', '위', '위', '크림', '올리고'],
 ['크림', '위', '그림', '그려서'],
 ['하나', '예술', '성', '띄우는', '그런'],
 ['제'

Word2Vec 훈련시키기

In [8]:
model = Word2Vec(sentences=tokenized_text,vector_size=100, window=5, min_count=5, workers=4, sg=0)

In [9]:
model_result = model.wv.most_similar('라테')
print(model_result)

[('위', 0.13887982070446014), ('생각', 0.13149002194404602), ('크리', 0.06408979743719101), ('그림', 0.01915229670703411), ('마트', 0.009391160681843758), ('커피', -0.05987628921866417)]


Word2Vec 모델 저장 & 로드

In [10]:
model.wv.save_word2vec_format('kor_w2v')
loaded_model = KeyedVectors.load_word2vec_format("kor_w2v")

In [11]:
# 모델 로드 하기 전의 결과와 동일하게
model_result = loaded_model.most_similar("라테")
print(model_result)

[('위', 0.13887982070446014), ('생각', 0.13149002194404602), ('크리', 0.06408979743719101), ('그림', 0.01915229670703411), ('마트', 0.009391160681843758), ('커피', -0.05987628921866417)]
